In [23]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import os

df = pd.read_csv("df_transactions.csv")
df.dtypes

tx_index            int64
tx_index.1          int64
timestamp           int64
type               object
amount              int64
nameOrig           object
oldbalanceOrig    float64
newbalanceOrig      int64
nameDest           object
oldbalanceDest      int64
newbalanceDest      int64
dtype: object

In [24]:
amount_received_per_nameDest = df.groupby("nameDest")["amount"].agg(['mean', 'count', 'sum'])
print(amount_received_per_nameDest)
amount_received_per_nameDest.sort_values("count", inplace=True, ascending=False)
top_address_transaction_count = amount_received_per_nameDest.iloc[0]["count"]
top_address_transaction_sum = amount_received_per_nameDest.iloc[0]["sum"]
top_address = amount_received_per_nameDest.index[0]
print(f"The top account {top_address} receive {int(top_address_transaction_count)} transactions which corresponds to a total transaction amount received of {top_address_transaction_sum}.")

amount_received_per_nameDest

                     mean  count         sum
nameDest                                    
C1000004082  3.765541e+08      6  2259324390
C1000004940  1.949234e+08     13  2534004050
C1000013769  4.772371e+08     13  6204082940
C100001587   1.560349e+08      9  1404313660
C1000015936  1.417475e+08     16  2267960190
...                   ...    ...         ...
M999998692   3.156540e+06      1     3156540
M99999900    3.426371e+07      1    34263710
M999999089   1.672552e+07      1    16725520
M999999543   1.936523e+07      1    19365230
M999999784   2.327350e+06      1     2327350

[2722364 rows x 3 columns]
The top account C52983754 receive 1372194 transactions which corresponds to a total transaction amount received of 13721940.0.


,mean,count,sum
nameDest,,,
C52983754,1.000000e+01,1372194,13721940
C1286084959,6.852119e+08,113,77428943310
C985934102,3.892008e+08,109,42422887980
C665576141,8.452322e+08,105,88749384380
C2083562754,5.203327e+08,102,53073938760
...,...,...,...
M1576070058,4.340330e+06,1,4340330
M1576069003,6.849950e+06,1,6849950
M1576065179,5.803000e+04,1,58030


In [25]:
# Adding the transaction error as a new column
df["new_balance_error"] = df["oldbalanceDest"] + df["amount"] - df["newbalanceDest"]
print("The following errors were found:")
print(df["new_balance_error"].unique())


The following errors were found:
[ 0 10  5]


In [38]:
# All transactions where the rounding error occured
inconsistent_new_balances = df[df["new_balance_error"] != 0].copy()
inconsistent_new_balances["nibbling_tx"] = np.nan
inconsistent_new_balances

,tx_index,tx_index.1,timestamp,type,amount,nameOrig,oldbalanceOrig,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,new_balance_error,nibbling_tx
1,2,2,1,TRANSFER,9839640,C1231006815,1.701360e+08,160296360,M1979787155,0,9839630,10,NaN
5,6,6,1,TRANSFER,11668140,C2048537720,4.155400e+07,29885855,M1230701703,0,11668135,5,NaN
7,8,8,1,TRANSFER,7817710,C90045638,5.386000e+07,46042290,M573487274,0,7817700,10,NaN
8,9,9,1,TRANSFER,7107770,C154988899,1.831950e+08,176087225,M408069119,0,7107765,5,NaN
9,10,10,1,TRANSFER,7861640,C1912850431,1.760872e+08,168225585,M633326333,0,7861635,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7734823,7734824,7734824,742,TRANSFER,63416990,C994950684,6.341699e+07,-5,C1662241365,438545215,501962200,5,NaN
7734826,7734827,7734827,743,TRANSFER,339682130,C2013999242,3.396821e+08,-5,C1850423904,0,339682125,5,NaN
7734827,7734828,7734828,743,TRANSFER,339682130,C786484425,3.396821e+08,-5,C776919290,382920110,722602235,5,NaN
7734830,7734831,7734831,743,TRANSFER,6311409280,C1162922333,6.311409e+09,0,C1365125890,351848635,6663257905,10,NaN


In [43]:
inconsistent_new_balances.groupby("new_balance_error").size()/inconsistent_new_balances.shape[0]

new_balance_error
5     0.593084
10    0.406916
dtype: float64

In [27]:
# All transactions to the suspicious address:
transactions_to_suspicious_account = df[df["nameDest"] == "C52983754"].copy()
transactions_to_suspicious_account["nibbled_tx"] = np.nan
transactions_to_suspicious_account.set_index("nameOrig", inplace=True, drop=False)
transactions_to_suspicious_account.dropna(axis=0, how="all", inplace=True)

In [28]:
# All transactions to the suspicious account have the exact same amount.
transactions_to_suspicious_account["amount"].unique()

array([10])

In [29]:
transactions_to_suspicious_account

,tx_index,tx_index.1,timestamp,type,amount,nameOrig,oldbalanceOrig,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,new_balance_error,nibbled_tx
nameOrig,,,,,,,,,,,,,
C1231006815,1,1,1,TRANSFER,10,C1231006815,1.701360e+08,170135990,C52983754,10,20,0,NaN
C90045638,7,7,1,TRANSFER,10,C90045638,5.386000e+07,53859990,C52983754,20,30,0,NaN
C249177573,14,14,1,TRANSFER,10,C249177573,2.077100e+07,20770990,C52983754,30,40,0,NaN
C1716932897,17,17,1,TRANSFER,10,C1716932897,1.012700e+07,10126990,C52983754,40,50,0,NaN
C761750706,21,21,1,TRANSFER,10,C761750706,4.500000e+05,449990,C52983754,50,60,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
C31496887,7734803,7734803,741,TRANSFER,10,C31496887,1.449453e+08,144945330,C52983754,13721900,13721910,0,NaN
C1970706589,7734809,7734809,742,TRANSFER,10,C1970706589,4.009058e+09,4009058380,C52983754,13721910,13721920,0,NaN
C1674778854,7734815,7734815,742,TRANSFER,10,C1674778854,5.465246e+07,54652450,C52983754,13721920,13721930,0,NaN


In [48]:
# Lets check if any addresses that have a rounding error also made a transaction to the suspicious account!
# We dont need all columns so lets simplify:
tx_sus_account = transactions_to_suspicious_account[["tx_index", "amount", "timestamp", "nameOrig"]].copy()
tx_sus_account.dropna(axis=0, how="all", inplace=True)
print(tx_sus_account)
for row_index, sus in tqdm(inconsistent_new_balances[:10000].iterrows()):
    # There should be exactly 1 nibbled tx for each rounding error.!
    try:
        nib_tx = tx_sus_account.loc[sus["nameOrig"]]
        if nib_tx["amount"] == sus["new_balance_error"]:
            # If the transaction has the same amount as the rounding error, its likely a nibbled tx!
            transactions_to_suspicious_account.loc[sus["nameOrig"], "nibbled_tx"] = row_index
            inconsistent_new_balances.loc[row_index, "nibbling_tx"] = nib_tx["tx_index"]
    except KeyError:
        pass


             tx_index  amount  timestamp     nameOrig
nameOrig                                             
C1231006815         1      10          1  C1231006815
C90045638           7      10          1    C90045638
C249177573         14      10          1   C249177573
C1716932897        17      10          1  C1716932897
C761750706         21      10          1   C761750706
...               ...     ...        ...          ...
C31496887     7734803      10        741    C31496887
C1970706589   7734809      10        742  C1970706589
C1674778854   7734815      10        742  C1674778854
C1226129332   7734820      10        742  C1226129332
C1162922333   7734830      10        743  C1162922333

[1372194 rows x 4 columns]


KeyboardInterrupt: 

In [46]:
print(transactions_to_suspicious_account[["tx_index", "timestamp", "amount", "nameOrig", "nameDest", "nibbled_tx"]][:10].to_latex())

\begin{tabular}{lrrrllr}
\toprule
{} &  tx\_index &  timestamp &  amount &     nameOrig &   nameDest &  nibbled\_tx \\
nameOrig    &           &            &         &              &            &             \\
\midrule
C1231006815 &         1 &          1 &      10 &  C1231006815 &  C52983754 &         1.0 \\
C90045638   &         7 &          1 &      10 &    C90045638 &  C52983754 &         7.0 \\
C249177573  &        14 &          1 &      10 &   C249177573 &  C52983754 &        14.0 \\
C1716932897 &        17 &          1 &      10 &  C1716932897 &  C52983754 &        17.0 \\
C761750706  &        21 &          1 &      10 &   C761750706 &  C52983754 &        21.0 \\
C2033524545 &        24 &          1 &      10 &  C2033524545 &  C52983754 &        24.0 \\
C20804602   &        27 &          1 &      10 &    C20804602 &  C52983754 &        27.0 \\
C1959239586 &        30 &          1 &      10 &  C1959239586 &  C52983754 &        30.0 \\
C1984094095 &        33 &          1 &      

In [45]:
transactions_to_suspicious_account[["tx_index", "timestamp", "amount", "nameOrig", "nameDest", "nibbled_tx"]]

,tx_index,timestamp,amount,nameOrig,nameDest,nibbled_tx
nameOrig,,,,,,
C1231006815,1,1,10,C1231006815,C52983754,1.0
C90045638,7,1,10,C90045638,C52983754,7.0
C249177573,14,1,10,C249177573,C52983754,14.0
C1716932897,17,1,10,C1716932897,C52983754,17.0
C761750706,21,1,10,C761750706,C52983754,21.0
...,...,...,...,...,...,...
C31496887,7734803,741,10,C31496887,C52983754,NaN
C1970706589,7734809,742,10,C1970706589,C52983754,NaN
C1674778854,7734815,742,10,C1674778854,C52983754,NaN


In [47]:
inconsistent_new_balances

,tx_index,tx_index.1,timestamp,type,amount,nameOrig,oldbalanceOrig,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,new_balance_error,nibbling_tx


In [33]:
print("Still unexplained rounding errors:")

inconsistent_new_balances[inconsistent_new_balances["nibbling_tx"].isna()]

Still unexplained rounding errors:


,tx_index,tx_index.1,timestamp,type,amount,nameOrig,oldbalanceOrig,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,new_balance_error,nibbling_tx
5,6,6,1,TRANSFER,11668140,C2048537720,4.155400e+07,29885855,M1230701703,0,11668135,5,NaN
8,9,9,1,TRANSFER,7107770,C154988899,1.831950e+08,176087225,M408069119,0,7107765,5,NaN
9,10,10,1,TRANSFER,7861640,C1912850431,1.760872e+08,168225585,M633326333,0,7861635,5,NaN
15,16,16,1,TRANSFER,2560740,C1648232591,5.070000e+06,2509255,M972865270,0,2560735,5,NaN
18,19,19,1,TRANSFER,4098780,C1026483832,5.032640e+08,499165215,M1635378213,0,4098775,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7734823,7734824,7734824,742,TRANSFER,63416990,C994950684,6.341699e+07,-5,C1662241365,438545215,501962200,5,NaN
7734826,7734827,7734827,743,TRANSFER,339682130,C2013999242,3.396821e+08,-5,C1850423904,0,339682125,5,NaN
7734827,7734828,7734828,743,TRANSFER,339682130,C786484425,3.396821e+08,-5,C776919290,382920110,722602235,5,NaN
7734830,7734831,7734831,743,TRANSFER,6311409280,C1162922333,6.311409e+09,0,C1365125890,351848635,6663257905,10,NaN


In [34]:
inconsistent_new_balances["new_balance_error"].sum()/1000

23721.87

In [35]:
inconsistent_new_balances["new_balance_error"].unique()

array([10,  5])

In [53]:
print(inconsistent_new_balances.dropna()[["tx_index", "timestamp", "amount", "nameOrig", "nameDest", "nibbling_tx"]][:10].to_latex(index=False))

\begin{tabular}{rrrllr}
\toprule
 tx\_index &  timestamp &     amount &     nameOrig &     nameDest &  nibbling\_tx \\
\midrule
        2 &          1 &    9839640 &  C1231006815 &  M1979787155 &          1.0 \\
        8 &          1 &    7817710 &    C90045638 &   M573487274 &          7.0 \\
       15 &          1 &    3099970 &   C249177573 &  M2096539129 &         14.0 \\
       18 &          1 &   11633760 &  C1716932897 &   M801569151 &         17.0 \\
       22 &          1 &    1563820 &   C761750706 &  M1731217984 &         21.0 \\
       25 &          1 &     671640 &  C2033524545 &   M473053293 &         24.0 \\
       28 &          1 &    1373430 &    C20804602 &  M1344519051 &         27.0 \\
       31 &          1 &    1065410 &  C1959239586 &   C515132998 &         30.0 \\
       34 &          1 &  311685890 &  C1984094095 &   C932583850 &         33.0 \\
       37 &          1 &    9478390 &  C1671590089 &    M58488213 &         36.0 \\
\bottomrule
\end{tabular}

